<h1>Preprocessing steps are listed below</h1>

* [Optionally eliminate case spesific non-important tweets](#Optionally-eliminate-case-spesific-non-important-tweets)
* [Remove mentions](#Remove-mentions)
* [Remove links](#Remove-links)
* [Remove Emojis](#Remove-Emojis)
* [Start zemberek](#Starting-zemberek)
    * Visit [here](https://www.kaggle.com/egebasturk1/yemeksepeti-sentiment-analysis/data) to get zemberek files 
* [Normalize tweets by using zemberek](#Normalize-tweets-by-using-zemberek)
* [Detect languages](#Detect-languages)
* [Eliminate non-turkish tweets](#Eliminate-non-turkish-tweets)
* [Remove hashtags](#Remove-hashtags)
* [Remove punctuations](#Remove-punctuations)
* [Strip numbers](#Strip-numbers)
* [Strip whitespaces](#Strip-whitespaces)
* [Correct accented turkish letters](#Correct-accented-turkish-letters)
* [Lower all letters including turkish](#Lower-all-characters-including-turkish)
* [Remove all non-turkish characters](#Remove-all-non-turkish-characters)
* [Remove short words](#Remove-short-words)
* [Remove stop words](#Remove-stop-words)
* [Lemmatize](#Lemmatize)
    * Visit [here](https://github.com/akoksal/Turkish-Lemmatizer/blob/master/lemmatizer.py)

In [1]:
import pandas as pd
import numpy as np

tweets_df = pd.read_excel("filtered_labeled_tweets_excel.xlsx", usecols=['Datetime', 'Text', 'rescue']) #.sample(frac=.1)
tweets_df = tweets_df.drop_duplicates(subset=['Text'], keep='last')
tweets_df

,Datetime,Text,rescue
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1
...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0


# Optionally eliminate case spesific non-important tweets 

In [17]:
tweets_df = tweets_df[((tweets_df['Text'].str.contains('deprem')) & (tweets_df['rescue'] == 0)) | (tweets_df['rescue'] == 1)]
tweets_df

,Datetime,Text,rescue,remove_ments
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1.0,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1.0,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1.0,10 yılı aşkın süredir travma/bellek üzerine ça...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1.0,"Değerli İzmirliler,\nDepremzedeler için iyi ni..."
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1.0,Minik Elif'in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...
52894,2020-10-30 12:00:32+00:00,tüm depremlerin nerde olursa olsun bursada his...,0.0,tüm depremlerin nerde olursa olsun bursada his...
52899,2020-10-30 12:00:20+00:00,Özellikle İzmir ve Kuşadası çok etkilenmiş gör...,0.0,Özellikle İzmir ve Kuşadası çok etkilenmiş gör...
52901,2020-10-30 12:00:16+00:00,"İzmir""de çok şiddetli bir deprem yaşadık. Geçm...",0.0,"İzmir""de çok şiddetli bir deprem yaşadık. Geçm..."
52903,2020-10-30 12:00:10+00:00,"Ege bölgesinde meydana gelen deprem İzmir, Man...",0.0,"Ege bölgesinde meydana gelen deprem İzmir, Man..."


In [2]:

#tweets_df = tweets_df[tweets_df['Text'].str.\
#          contains('acil|yayalım|yayalim|elden ele|paylaşalım|paylaşalim|paylasalim|paylasalım')] # rescue candidate words
#tweets_df

,Datetime,Text
17,2020-10-23 17:50:59+03:00,"@baskanzorluoglu Başkanım, olası bir deprem iç..."
68,2020-10-23 19:39:06+03:00,"Değerli arkadaşlar, nasıl ki ağır bir deprem b..."
97,2020-10-23 20:20:29+03:00,"Deprem, sel vb. doğal afetlerin yanı sıra topl..."
157,2020-10-23 21:50:44+03:00,@ibbhaberleri Osuruktan teyyare anketi yapıyor...
277,2020-10-24 01:27:00+03:00,@bikapiacildi Bilimsel bir gerçek sadece. Oras...
...,...,...
1508588,2021-09-30 11:33:35+03:00,İLAVE OLARAK\nsiyasal islam\ntarikatlar\ndin e...
1508599,2021-09-30 11:45:21+03:00,#Deprem ve acil durumlarda iletişimin kesilmem...
1508670,2021-09-30 13:39:16+03:00,@ynrzl @ekin_ozsoy İstanbulun en önemli sorunu...
1509182,2021-09-30 22:42:51+03:00,Bu hayatin ucaklarin acil cikis kapisinda otur...


# Remove mentions

In [2]:
tweets_df['remove_ments'] = tweets_df['Text'].str.replace('(@\w+.*?)'," ")
#tweets_df['processed1'].to_csv('processed1withoutmentions.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['remove_ments'], keep='last')
tweets_df

<ipython-input-2-413d201f80c6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets_df['remove_ments'] = tweets_df['Text'].str.replace('(@\w+.*?)'," ")


,Datetime,Text,rescue,remove_ments
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni..."
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a..."


# Remove links

In [3]:
tweets_df['remove_links'] = tweets_df['remove_ments'].str.replace(r'http\S+|www.\S+', ' ', case=False)
#tweets_df['remove_links'].to_csv('processed2withoutlinks.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['remove_links'], keep='last')
tweets_df

<ipython-input-3-8f989f45e5aa>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets_df['remove_links'] = tweets_df['remove_ments'].str.replace(r'http\S+|www.\S+', ' ', case=False)
<ipython-input-3-8f989f45e5aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['remove_links'] = tweets_df['remove_ments'].str.replace(r'http\S+|www.\S+', ' ', case=False)


,Datetime,Text,rescue,remove_ments,remove_links
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni..."
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS..."
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a..."


# Remove Emojis

In [4]:

import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', str(data))
tweets_df['remove_emojis'] = tweets_df['remove_links'].apply(remove_emojis)
#tweets_df['processed3'].to_csv('processed3withoutemojis.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['remove_emojis'], keep='last')
tweets_df

<ipython-input-4-a7146719dfc6>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['remove_emojis'] = tweets_df['remove_links'].apply(remove_emojis)


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni..."
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS..."
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a..."


# Starting zemberek

In [10]:
### START ZEMBEREK 
#
# Requirements: zemberek-full.jar, lm.2gram.slm and normalization folder contains ascii-map, lookup-from-graph and split files
# visit https://www.kaggle.com/egebasturk1/yemeksepeti-sentiment-analysis/data

import jpype as jp

ZEMBEREK_PATH = r'zemberek/zemberek-full.jar'
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

In [21]:
TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')
#TurkishSpellChecker = jp.JClass('zemberek.normalization.TurkishSpellChecker')
TurkishSentenceNormalizer = jp.JClass('zemberek.normalization.TurkishSentenceNormalizer')
Paths = jp.JClass('java.nio.file.Paths')

lookupRoot = Paths.get(r'zemberek/normalization')
lmPath = Paths.get(r'zemberek/lm.2gram.slm')
morphology = TurkishMorphology.createWithDefaults()
normalizer = TurkishSentenceNormalizer(morphology, lookupRoot, lmPath)

# Normalize tweets by using zemberek

In [22]:
### Normalize the tweets

from tqdm import tqdm
tqdm.pandas()

def normalization(tweet):
    return " " if len(tweet) == 0 or tweet.isspace() else str(normalizer.normalize(jp.JString(tweet)))

tweets_df['normalize'] = tweets_df['remove_emojis'].progress_apply(normalization)
#tweets_df['processed4'].to_csv('processed4normalized.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['normalize'], keep='last')
tweets_df

C:\Users\ali\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|███████████████████████████████████████████████████████████████████████████| 13346/13346 [00:22<00:00, 603.13it/s]
<ipython-input-22-9324536ab812>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['normalize'] = tweets_df['remove_emojis'].progress_apply(normalization)


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,normalize
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1.0,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,merhaba . biz mansuroğlu mahallesinde oturan b...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1.0,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1.0,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma / bellek üzerine ...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1.0,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","değerli izmirliler , depremzedeler için iyi ni..."
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1.0,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,minik elif'in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...
52894,2020-10-30 12:00:32+00:00,tüm depremlerin nerde olursa olsun bursada his...,0.0,tüm depremlerin nerde olursa olsun bursada his...,tüm depremlerin nerde olursa olsun bursada his...,tüm depremlerin nerde olursa olsun bursada his...,tüm depremlerin nerde olursa olsun bursa da hi...
52899,2020-10-30 12:00:20+00:00,Özellikle İzmir ve Kuşadası çok etkilenmiş gör...,0.0,Özellikle İzmir ve Kuşadası çok etkilenmiş gör...,Özellikle İzmir ve Kuşadası çok etkilenmiş gör...,Özellikle İzmir ve Kuşadası çok etkilenmiş gör...,özellikle izmir ve kuşadası çok etkilenmiş gör...
52901,2020-10-30 12:00:16+00:00,"İzmir""de çok şiddetli bir deprem yaşadık. Geçm...",0.0,"İzmir""de çok şiddetli bir deprem yaşadık. Geçm...","İzmir""de çok şiddetli bir deprem yaşadık. Geçm...","İzmir""de çok şiddetli bir deprem yaşadık. Geçm...","izmir "" de çok şiddetli bir deprem yaşadık . g..."
52903,2020-10-30 12:00:10+00:00,"Ege bölgesinde meydana gelen deprem İzmir, Man...",0.0,"Ege bölgesinde meydana gelen deprem İzmir, Man...","Ege bölgesinde meydana gelen deprem İzmir, Man...","Ege bölgesinde meydana gelen deprem İzmir, Man...","ege bölgesinde meydana gelen deprem izmir , ma..."


# Detect languages

In [27]:
### Detect languages

import langid

tweets_df['languages'] = tweets_df['normalize'].progress_apply(lambda s: langid.classify(s)[0])
tweets_df

100%|███████████████████████████████████████████████████████████████████████████| 50499/50499 [02:49<00:00, 297.81it/s]
<ipython-input-27-b13d79bfd78b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['languages'] = tweets_df['normalize'].progress_apply(lambda s: langid.classify(s)[0])


,Datetime,Text,remove_ments,remove_links,remove_emojis,normalize,languages
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,merhaba . biz mansuroğlu mahallesinde oturan b...,tr
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,tr
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma / bellek üzerine ...,tr
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","değerli izmirliler , depremzedeler için iyi ni...",tr
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,minik elif'in hayatı boyunca tüm eğitim masraf...,tr
...,...,...,...,...,...,...,...
53768,2020-10-30 12:00:03+00:00,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#sondakika ege denizi merkezli 6.6 büyüklüğünd...,tr
53769,2020-10-30 12:00:02+00:00,¡RAFAtatatata MIR! 🔫💥💥💥\n\n#LaLigaSantander \n...,¡RAFAtatatata MIR! 🔫💥💥💥\n\n#LaLigaSantander \n...,¡RAFAtatatata MIR! 🔫💥💥💥\n\n#LaLigaSantander \n...,¡RAFAtatatata MIR! \n\n#LaLigaSantander \n#Ha...,¡rafatatatata mir ! #lalığasantander #hayquevi...,tr
53770,2020-10-30 12:00:01+00:00,"""Karşınızda artık eski Reyyan yok!""\nHercai ye...","""Karşınızda artık eski Reyyan yok!""\nHercai ye...","""Karşınızda artık eski Reyyan yok!""\nHercai ye...","""Karşınızda artık eski Reyyan yok!""\nHercai ye...",""" karşınızda artık eski reyyan yok ! "" hercai ...",tr
53771,2020-10-30 12:00:01+00:00,Depremi İzmir Urla’da çok şiddetli ve uzun his...,Depremi İzmir Urla’da çok şiddetli ve uzun his...,Depremi İzmir Urla’da çok şiddetli ve uzun his...,Depremi İzmir Urla’da çok şiddetli ve uzun his...,depremi izmir urla’da çok şiddetli ve uzun his...,tr


# Eliminate non-turkish tweets

In [28]:

tweets_df = tweets_df[tweets_df['languages'] == 'tr']
tweets_df

,Datetime,Text,remove_ments,remove_links,remove_emojis,normalize,languages
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,merhaba . biz mansuroğlu mahallesinde oturan b...,tr
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,tr
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma / bellek üzerine ...,tr
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","değerli izmirliler , depremzedeler için iyi ni...",tr
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,minik elif'in hayatı boyunca tüm eğitim masraf...,tr
...,...,...,...,...,...,...,...
53768,2020-10-30 12:00:03+00:00,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#SONDAKİKA\nEge Denizi merkezli 6.6 büyüklüğün...,#sondakika ege denizi merkezli 6.6 büyüklüğünd...,tr
53769,2020-10-30 12:00:02+00:00,¡RAFAtatatata MIR! 🔫💥💥💥\n\n#LaLigaSantander \n...,¡RAFAtatatata MIR! 🔫💥💥💥\n\n#LaLigaSantander \n...,¡RAFAtatatata MIR! 🔫💥💥💥\n\n#LaLigaSantander \n...,¡RAFAtatatata MIR! \n\n#LaLigaSantander \n#Ha...,¡rafatatatata mir ! #lalığasantander #hayquevi...,tr
53770,2020-10-30 12:00:01+00:00,"""Karşınızda artık eski Reyyan yok!""\nHercai ye...","""Karşınızda artık eski Reyyan yok!""\nHercai ye...","""Karşınızda artık eski Reyyan yok!""\nHercai ye...","""Karşınızda artık eski Reyyan yok!""\nHercai ye...",""" karşınızda artık eski reyyan yok ! "" hercai ...",tr
53771,2020-10-30 12:00:01+00:00,Depremi İzmir Urla’da çok şiddetli ve uzun his...,Depremi İzmir Urla’da çok şiddetli ve uzun his...,Depremi İzmir Urla’da çok şiddetli ve uzun his...,Depremi İzmir Urla’da çok şiddetli ve uzun his...,depremi izmir urla’da çok şiddetli ve uzun his...,tr


# Remove hashtags

In [5]:

if 'normalize' in tweets_df.columns:
    tweets_df['remove_hashtags'] = tweets_df['normalize'].str.replace('(#\w+.*?)'," ")
else:
    tweets_df['remove_hashtags'] = tweets_df['remove_emojis'].str.replace('(#\w+.*?)'," ")
#tweets_df['processed5'].to_csv('processed5withouthashtags.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['remove_hashtags'], keep='last')
tweets_df

<ipython-input-5-9ae2ed68a33c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets_df['remove_hashtags'] = tweets_df['remove_emojis'].str.replace('(#\w+.*?)'," ")
<ipython-input-5-9ae2ed68a33c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['remove_hashtags'] = tweets_df['remove_emojis'].str.replace('(#\w+.*?)'," ")


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni..."
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","\nEGE DENIZI \n30.10.2020, 14:51:25 TSİ\nBüy..."
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a..."


# Remove punctuations

In [6]:

import gensim.parsing.preprocessing as gsp
import string

tweets_df['remove_punct'] = tweets_df['remove_hashtags'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation, ' '))))
#tweets_df['processed6'].to_csv('processed6withoutpunc.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['remove_punct'], keep='last')
tweets_df

C:\Users\ali\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
<ipython-input-6-0acb346d87aa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['remove_punct'] = tweets_df['remove_hashtags'].apply(lambda x: x.translate(str.maketrans(dict.fromkeys(string.punctuation, ' '))))


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","\nEGE DENIZI \n30.10.2020, 14:51:25 TSİ\nBüy...",\nEGE DENIZI \n30 10 2020 14 51 25 TSİ\nBüy...
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",AFAD merkez üssü Ege Denizi nin Seferihisar a...


# Strip whitespaces

In [7]:

#tweets_df['processed_6'] = tweets_df['processed_5'].apply(gsp.strip_punctuation)
tweets_df['strip_spaces'] = tweets_df['remove_punct'].apply(gsp.strip_multiple_whitespaces)
#tweets_df['processed7'].to_csv('processed7withoutspaces.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['strip_spaces'], keep='last')
tweets_df

<ipython-input-7-c239757b19bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['strip_spaces'] = tweets_df['remove_punct'].apply(gsp.strip_multiple_whitespaces)


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct,strip_spaces
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...,Değerli İzmirliler Depremzedeler için iyi niye...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesinde...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","\nEGE DENIZI \n30.10.2020, 14:51:25 TSİ\nBüy...",\nEGE DENIZI \n30 10 2020 14 51 25 TSİ\nBüy...,EGE DENIZI 30 10 2020 14 51 25 TSİ Büyüklük 6...
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",AFAD merkez üssü Ege Denizi nin Seferihisar a...,AFAD merkez üssü Ege Denizi nin Seferihisar aç...


# Strip numbers

In [8]:

tweets_df['strip_numbers'] = tweets_df['strip_spaces'].apply(gsp.strip_numeric)
#tweets_df['processed8'].to_csv('processed8withoutnums.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['strip_numbers'], keep='last')
tweets_df

<ipython-input-8-d8cf15e3c4a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['strip_numbers'] = tweets_df['strip_spaces'].apply(gsp.strip_numeric)


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct,strip_spaces,strip_numbers
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,yılı aşkın süredir travma bellek üzerine çalı...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","\nEGE DENIZI \n30.10.2020, 14:51:25 TSİ\nBüy...",\nEGE DENIZI \n30 10 2020 14 51 25 TSİ\nBüy...,EGE DENIZI 30 10 2020 14 51 25 TSİ Büyüklük 6...,EGE DENIZI TSİ Büyüklük Derinlik km
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...
5049,2020-10-30 12:00:45+00:00,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",0,"AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...","AFAD, merkez üssü Ege Denizi'nin Seferihisar a...",AFAD merkez üssü Ege Denizi nin Seferihisar a...,AFAD merkez üssü Ege Denizi nin Seferihisar aç...,AFAD merkez

# Correct accented turkish letters

In [9]:

def correct_old_characters(self):
    self = re.sub(r"Â", "A", self)
    self = re.sub(r"Î", "I", self)
    self = re.sub(r"î", "ı", self)
    self = re.sub(r"â", "a", self)
    self = re.sub(r"û", "u", self)
    self = re.sub(r"Û", "U", self)
    return self

tweets_df['correct_old_chars'] = tweets_df['strip_numbers'].apply(lambda x: correct_old_characters(x))
#tweets_df['processed9'].to_csv('processed9withoutaccented.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['correct_old_chars'], keep='last')
tweets_df

<ipython-input-9-7a6a61437c2d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['correct_old_chars'] = tweets_df['strip_numbers'].apply(lambda x: correct_old_characters(x))


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct,strip_spaces,strip_numbers,correct_old_chars
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","\nEGE DENIZI \n30.10.2020, 14:51:25 TSİ\nBüy...",\nEGE DENIZI \n30 10 2020 14 51 25 TSİ\nBüy...,EGE DENIZI 30 10 2020 14 51 25 TSİ Büyüklük 6...,EGE DENIZI TSİ Büyüklük Derinlik km,EGE DENIZI TSİ Büyüklük Derinlik km
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril...,Az önce yaşanan deprem nedeniyle tüm hemşehril.

# Lower all characters including turkish

In [10]:

def lower(self):
    self = re.sub(r"İ", "i", self)
    self = re.sub(r"I", "ı", self)
    self = re.sub(r"Ç", "ç", self)
    self = re.sub(r"Ş", "ş", self)
    self = re.sub(r"Ü", "ü", self)
    self = re.sub(r"Ğ", "ğ", self)
    self = self.lower() # for the rest use default lower
    return self

tweets_df['lower_trs'] = tweets_df['correct_old_chars'].apply(lambda x: lower(x))
#tweets_df['processed10'].to_csv('processed10lower.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['lower_trs'], keep='last')
tweets_df

<ipython-input-10-51c807b2c186>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['lower_trs'] = tweets_df['correct_old_chars'].apply(lambda x: lower(x))


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct,strip_spaces,strip_numbers,correct_old_chars,lower_trs
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,minik elif in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,ege denizi nde izmir in seferihisar ilçesinde...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","\nEGE DENIZI \n30.10.2020, 14:51:25 TSİ\nBüy...",\nEGE DENIZI \n30 10 2020 14 51 25 TSİ\nBüy...,EGE DENIZI 30 10 2020 14 51 25 TSİ Büyüklük 6...,EGE DENIZI TSİ Büyüklük Derinlik km,EGE DENIZI TSİ Büyüklük Derinlik km,ege denızı tsi büyüklük derinlik km
5048,2020-10-30 12:01:31+00:00,Az önce yaşanan deprem nedeniyle tüm hemşehril...,0,Az önce yaşanan deprem nedeniyle tüm hemşehril...

# Remove all non-turkish characters

In [11]:

whitelist = set('abcçdefgğhıijklmnoöpqrsştuüvwxyz ABCÇDEFGĞHIİJKLMNOÖPQRSŞTUÜVWXYZ')

tweets_df['remove_nt'] = tweets_df['lower_trs'].apply(lambda x: ''.join(filter(whitelist.__contains__, x)))
#tweets_df['processed11'].to_csv('processed11withoutnonturkish.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['remove_nt'], keep='last')
tweets_df

<ipython-input-11-48f9bd8257d0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['remove_nt'] = tweets_df['lower_trs'].apply(lambda x: ''.join(filter(whitelist.__contains__, x)))


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct,strip_spaces,strip_numbers,correct_old_chars,lower_trs,remove_nt
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,minik elif in hayatı boyunca tüm eğitim masraf...,minik elif in hayatı boyunca tüm eğitim masraf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,ege denizi nde izmir in seferihisar ilçesinde...,ege denizi nde izmir in seferihisar ilçesinde...
5047,2020-10-30 12:02:09+00:00,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,0,#DEPREM\nEGE DENIZI https://t.co/291sqKc2Hx\n3...,"#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","#DEPREM\nEGE DENIZI \n30.10.2020, 14:51:25 TS...","\nEGE DENIZI \n30.10.2020, 14:51:25 TSİ\nBüy...",\nEGE DENIZI \n30 10 202

# Remove short words

In [12]:

tweets_df['remove_shorts'] = tweets_df['remove_nt'].apply(gsp.strip_short)
#tweets_df['processed12'].to_csv('processed12withoutshorts.csv', header=False, index=False)
tweets_df = tweets_df.drop_duplicates(subset=['remove_shorts'], keep='last')
tweets_df

<ipython-input-12-022d4d1322f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['remove_shorts'] = tweets_df['remove_nt'].apply(gsp.strip_short)


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct,strip_spaces,strip_numbers,correct_old_chars,lower_trs,remove_nt,remove_shorts
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalış...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,minik elif in hayatı boyunca tüm eğitim masraf...,minik elif in hayatı boyunca tüm eğitim masraf...,minik elif hayatı boyunca tüm eğitim masraflar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5046,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,Ege Denizi nde İzmir in Seferihisar ilçesinde...,ege denizi nde izmir in seferihisar ilçesinde...,ege denizi nde izmir in seferihisar ilçesinde...,ege denizi nde izmir seferihisar ilçesind

In [13]:
for index, rows in tweets_df.sample(10).iterrows():
    for i, j in enumerate(rows):
        print(rows.index[i], '->',j)

Datetime -> 2020-10-30 12:10:57+00:00
Text -> Alaska'da 2018 yılında meydana gelen 7.1 şiddetindeki depremde, öğrencilerin gösterdiği refleks.
İşte bunlar hep eğitim announceemoji 

#Deprem  imagelink 
rescue -> 0
remove_ments -> Alaska'da 2018 yılında meydana gelen 7.1 şiddetindeki depremde, öğrencilerin gösterdiği refleks.
İşte bunlar hep eğitim announceemoji 

#Deprem  imagelink 
remove_links -> Alaska'da 2018 yılında meydana gelen 7.1 şiddetindeki depremde, öğrencilerin gösterdiği refleks.
İşte bunlar hep eğitim announceemoji 

#Deprem  imagelink 
remove_emojis -> Alaska'da 2018 yılında meydana gelen 7.1 şiddetindeki depremde, öğrencilerin gösterdiği refleks.
İşte bunlar hep eğitim announceemoji 

#Deprem  imagelink 
remove_hashtags -> Alaska'da 2018 yılında meydana gelen 7.1 şiddetindeki depremde, öğrencilerin gösterdiği refleks.
İşte bunlar hep eğitim announceemoji 

   imagelink 
remove_punct -> Alaska da 2018 yılında meydana gelen 7 1 şiddetindeki depremde  öğrencilerin gösterd

In [15]:
tweets_df[['remove_shorts']].to_csv('preprocessed_tweets.csv', index=False)

# Remove stop words

In [14]:
##

import nltk
nltk.download('punkt')

def remove_stopwords(text): 
    stop_words = ['çok', 'at', 'in', 'im', 'acaba','acep','allah','adamakıllı','adeta','ait','altmýþ',
                               'altmış','altý',
                  'altı','ama',
                       'amma','anca','ancak','arada','artýk','aslında','aynen','ayrıca','az','açıkça','açıkçası',
                       'bana','bari','bazen','bazý','bazı','başkası','baţka','belki','ben','benden','beni','benim',
                       'beri','beriki','beþ','beş','beţ','bilcümle','bile','bin','binaen','binaenaleyh','bir','biraz',
                       'birazdan','birbiri','birden','birdenbire','biri','birice','birileri','birisi','birkaç',
                       'birkaçı','birkez','birlikte','birçok','birçoğu','birþey','birþeyi','birşey','birşeyi','birţey',
                       'bitevi','biteviye','bittabi','biz','bizatihi','bizce','bizcileyin','bizden','bize','bizi','bizim',
                       'bizimki','bizzat','boşuna','bu','buna','bunda','bundan','bunlar','bunları','bunların','bunu','bunun',
                       'buracıkta','burada','buradan','burası','böyle','böylece','böylecene','böylelikle','böylemesine',
                       'böylesine','büsbütün','bütün','cuk','cümlesi','da','daha','dahi','dahil','dahilen','daima','dair',
                       'dayanarak','de','defa','dek','demin','demincek','deminden','denli','derakap','derhal','derken',
                       'deđil','değil','değin','diye','diđer','diğer','diğeri','doksan','dokuz','dolayı','dolayısıyla',
                       'doğru','dört','edecek','eden','ederek','edilecek','ediliyor','edilmesi','ediyor','elbet',
                       'elbette','elli','emme','en','enikonu','epey','epeyce','epeyi','esasen','esnasında','etmesi',
                       'etraflı','etraflıca','etti','ettiği','ettiğini','evleviyetle','evvel','evvela','evvelce',
                       'evvelden','evvelemirde','evveli','eđer','eğer','fakat','filanca','gah','gayet','gayetle','gayri',
                       'gayrı','gelgelelim','gene','gerek','gerçi','geçende','geçenlerde','gibi','gibilerden','gibisinden',
                       'gine','göre','gırla','hakeza','halbuki','halen','halihazırda','haliyle','handiyse','hangi','hangisi',
                       'hani','hariç','hasebiyle','hasılı','hatta','hele','hem','henüz','hep','hepsi','her',
                       'herhangi','herkes','herkesin','hiç','hiçbir','hiçbiri','hoş','hulasaten','iken','iki',
                       'ila','ile','ilen','ilgili','ilk','illa','illaki','imdi','indinde','inen','insermi','ise',
                       'ister','itibaren','itibariyle','itibarıyla','iyi','iyice','iyicene','için','iş','işte',
                       'iţte','kadar','kaffesi','kah','kala','kanýmca','karşın','katrilyon','kaynak','kaçı','kelli',
                       'kendi','kendilerine','kendini','kendisi','kendisine','kendisini','kere','kez','keza','kezalik',
                       'keşke','keţke','ki','kim','kimden','kime','kimi','kimisi','kimse','kimsecik','kimsecikler',
                       'külliyen','kýrk','kýsaca','kırk','kısaca','lakin','leh','lütfen','maada','madem','mademki',
                       'mamafih','mebni','međer','meğer','meğerki','meğerse','milyar','milyon','mu','mü','mý','mı',
                       'nasýl','nasıl','nasılsa','nazaran','naşi','ne','neden','nedeniyle','nedenle','nedense',
                       'nerde','nerden','nerdeyse','nere','nerede','nereden','neredeyse','neresi','nereye',
                       'netekim','neye','neyi','neyse','nice','nihayet','nihayetinde','nitekim','niye','niçin',
                       'o','olan','olarak','oldu','olduklarını','oldukça','olduğu','olduğunu','olmadı',
                       'olmadığı','olmak','olması','olmayan','olmaz','olsa','olsun','olup','olur','olursa','oluyor',
                       'on','ona','onca','onculayın','onda','ondan','onlar','onlardan','onlari','onlarýn','onları',
                       'onların','onu','onun','oracık','oracıkta','orada','oradan','oranca','oranla','oraya','otuz',
                       'oysa','oysaki','pek','pekala','peki','pekçe','peyderpey','rağmen','sadece','sahi','sahiden',
                       'sana','sanki','sekiz','seksen','sen','senden','seni','senin','siz','sizden','sizi','sizin',
                       'sonra','sonradan','sonraları','sonunda','tabii','tam','tamam','tamamen','tamamıyla','tarafından',
                       'tek','trilyon','tüm','var','vardı','vasıtasıyla','ve','velev','velhasıl','velhasılıkelam','veya',
                       'veyahut','ya','yahut','yakinen','yakında','yakından','yakınlarda','yalnız','yalnızca','yani',
                       'yapacak','yapmak','yaptı','yaptıkları','yaptığı','yaptığını','yapılan','yapılması','yapıyor',
                       'yedi','yeniden','yenilerde','yerine','yetmiþ','yetmiş','yetmiţ','yine','yirmi','yok','yoksa',
                       'yoluyla','yüz','yüzünden','zarfında','zaten','zati','zira','çabuk','çabukça','çeşitli',
                       'çok','çokları','çoklarınca','çokluk','çoklukla','çokça','çoğu','çoğun','çoğunca','çoğunlukla',
                       'çünkü','öbür','öbürkü','öbürü','önce','önceden','önceleri','öncelikle','öteki','ötekisi','öyle',
                       'öylece','öylelikle','öylemesine','öz','üzere','üç','þey','þeyden','þeyi','þeyler','þu','þuna',
                       'þunda','þundan','þunu','şayet','şey','şeyden','şeyi','şeyler','şu','şuna','şuncacık','şunda',
                       'şundan','şunlar','şunları','şunu','şunun','şura','şuracık','şuracıkta','şurası','şöyle',
                       'ţayet','ţimdi','ţu','ţöyle', 'hala', 'yer', 'güzel', 'büyük']
    stop_words = ['a','acaba','altı','altmış','ama','ancak','arada','artık','asla','aslında','aslında','ayrıca',
                  'az','bana','bazen','bazı','bazıları','belki','ben','benden','beni','benim','beri','beş',
                  'bile','bilhassa','bin','bir','biraz','birçoğu','birçok','biri','birisi','birkaç','birşey',
                  'biz','bizden','bize','bizi','bizim','böyle','böylece','bu','buna','bunda','bundan','bunlar',
                  'bunları','bunların','bunu','bunun','burada','bütün','çoğu','çoğunu','çok','çünkü','da',
                  'daha','dahi','dan','de','defa','değil','diğer','diğeri','diğerleri','diye','doksan','dokuz',
                  'dolayı','dolayısıyla','dört','e','edecek','eden','ederek','edilecek','ediliyor','edilmesi',
                  'ediyor','eğer','elbette','elli','en','etmesi','etti','ettiği','ettiğini','fakat','falan',
                  'filan','gene','gereği','gerek','gibi','göre','hala','halde','halen','hangi','hangisi',
                  'hani','hatta','hem','henüz','hep','hepsi','her','herhangi','herkes','herkese','herkesi',
                  'herkesin','hiç','hiçbir','hiçbiri','i','ı','için','içinde','iki','ile','ilgili','ise',
                  'işte','itibaren','itibariyle','kaç','kadar','karşın','kendi','kendilerine','kendine',
                  'kendini','kendisi','kendisine','kendisini','kez','ki','kim','kime','kimi','kimin',
                  'kimisi','kimse','kırk','madem','mi','mı','milyar','milyon','mu','mü','nasıl','ne',
                  'neden','nedenle','nerde','nerede','nereye','neyse','niçin','nin','nın','niye','nun',
                  'nün','o','öbür','olan','olarak','oldu','olduğu','olduğunu','olduklarını','olmadı',
                  'olmadığı','olmak','olması','olmayan','olmaz','olsa','olsun','olup','olur','olur','olursa',
                  'oluyor','on','ön','ona','önce','ondan','onlar','onlara','onlardan','onları','onların',
                  'onu','onun','orada','öte','ötürü','otuz','öyle','oysa','pek','rağmen','sana','sanki',
                  'sanki','şayet','şekilde','sekiz','seksen','sen','senden','seni','senin','şey','şeyden',
                  'şeye','şeyi','şeyler','şimdi','siz','siz','sizden','sizden','size','sizi','sizi',
                  'sizin','sizin','sonra','şöyle','şu','şuna','şunları','şunu','ta','tabii','tam',
                  'tamam','tamamen','tarafından','trilyon','tüm','tümü','u','ü','üç','un','ün','üzere',
                  'var','vardı','ve','veya','ya','yani','yapacak','yapılan','yapılması','yapıyor','yapmak',
                  'yaptı','yaptığı','yaptığını','yaptıkları','ye','yedi','yerine','yetmiş','yi','yı','yine',
                  'yirmi','yoksa','yu','yüz','zaten','zira','zxtest']
    
    stop_words = ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'bir', 'birkaç', 'birşey', 'biz',
                  'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'den', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem',
                  'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü',
                  'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey',
                  'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'dan']
    word_tokens = nltk.word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    return ' '.join(filtered_text)

tweets_df['remove_stops'] = tweets_df['remove_shorts'].apply(remove_stopwords)
#tweets_df['processed13'].to_csv('processed13withoutstops.csv', header=False, index=False)
tweets_df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ali\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-14-55f7f478a161>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['remove_stops'] = tweets_df['remove_shorts'].apply(remove_stopwords)


,Datetime,Text,rescue,remove_ments,remove_links,remove_emojis,remove_hashtags,remove_punct,strip_spaces,strip_numbers,correct_old_chars,lower_trs,remove_nt,remove_shorts,remove_stops
0,2020-11-03 17:42:44+00:00,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,1,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba. Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bi...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,Merhaba Biz Mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...,merhaba biz mansuroğlu mahallesinde oturan bir...,merhaba mansuroğlu mahallesinde oturan çiftiz ...
1,2020-11-03 13:11:49+00:00,Depremde evlerini kaybeden vatandaşlarımızla o...,1,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,Depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...,depremde evlerini kaybeden vatandaşlarımızla o...
2,2020-11-03 07:35:38+00:00,10 yılı aşkın süredir travma/bellek üzerine ça...,1,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma/bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,10 yılı aşkın süredir travma bellek üzerine ça...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalı...,yılı aşkın süredir travma bellek üzerine çalış...,yılı aşkın süredir travma bellek üzerine çalış...
3,2020-11-02 09:26:46+00:00,"Değerli İzmirliler,\nDepremzedeler için iyi ni...",1,"Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...","Değerli İzmirliler,\nDepremzedeler için iyi ni...",Değerli İzmirliler \nDepremzedeler için iyi ni...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,Değerli İzmirliler Depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...,değerli izmirliler depremzedeler için iyi niye...,değerli izmirliler depremzedeler iyi niyetle b...
4,2020-11-02 09:23:50+00:00,Minik Elif'in hayatı boyunca tüm eğitim masraf...,1,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif'in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,Minik Elif in hayatı boyunca tüm eğitim masraf...,minik elif in hayatı boyunca tüm eğitim masraf...,minik elif in hayatı boyunca tüm eğitim masraf...,minik elif hayatı boyunca tüm eğitim masraflar...,minik elif hayatı boyunca eğitim masraflarını ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546,2020-10-30 12:02:33+00:00,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,0,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,#DEPREM: Ege Denizi'nde İzmir'in Seferihisar i...,: Ege Denizi'nde İzmir'in Seferihisar ilçesin...,Ege Denizi nde İzmir in Seferihisar ilçesin...,Ege Denizi nde İzmi

# Lemmatize

In [20]:

import re
import json
import pickle
import sys
import nltk

def check(root, suffix, guess, action):
    if action == "unsuz yumusamasi":
        return len(suffix)>0 and suffix[0] in ["a","e","ı","i","o","ö","u","ü"] and checkSuffixValidation(suffix)[0]
    if action == "unlu daralmasi":
        if guess=="demek" and checkSuffixValidation(suffix)[0]:
            return True
        if guess=="yemek" and checkSuffixValidation(suffix)[0]:
            return True
        
        if suffix.startswith("yor"):
            lastVowel = ""
            for letter in reversed(guess[:-3]):
                if letter in ["a","e","ı","i","o","ö","u","ü"]:
                    lastVowel = letter
                    break
            if lastVowel in ["a","e"] and checkSuffixValidation(suffix)[0]:
                return True
        return False
    if action == "fiil" or action == "olumsuzluk eki":
        return checkSuffixValidation(suffix)[0] and not ((root.endswith("la") or (root.endswith("le"))) and suffix.startswith("r"))
    if action == "unlu dusmesi":
        count = 0
        for letter in guess:
            if letter in ["a","e","ı","i","o","ö","u","ü"]:
                count+=1
                lastVowel = letter
        if checkSuffixValidation(suffix)[0] and count==2 and (lastVowel in ["ı","i","u","ü"]) and (len(suffix)>0 and suffix[0] in ["a","e","ı","i","o","ö","u","ü"]):
            if lastVowel == "ı":
                return suffix[0] in ["a","ı"]
            elif lastVowel == "i":
                return suffix[0] in ["e","i"]
            elif lastVowel == "u":
                return suffix[0] in ["a","u"]
            elif lastVowel == "ü":
                return suffix[0] in ["e","ü"]
        return False
    return True

def findPos(kelime,revisedDict):
    l = []
    if "'" in kelime:
        l.append([kelime[:kelime.index("'")]+"_1","tirnaksiz",kelime])
    mid = []
    for i in range(len(kelime)):
        guess = kelime[:len(kelime)-i]
        suffix = kelime[len(kelime)-i:]
        ct = 1
            
        while guess+"_"+str(ct) in revisedDict:
            if check(guess, suffix, revisedDict[guess+"_"+str(ct)][1], revisedDict[guess+"_"+str(ct)][0]):
                guessList = (revisedDict[guess+"_"+str(ct)])
                while guessList[0] not in ["kok","fiil","olumsuzluk"]:
                    guessList = revisedDict[guessList[1]]
                mid.append([guessList[1], revisedDict[guess+"_"+str(ct)][0],guess+"_"+str(ct)])
            ct = ct+1
            
    temp = []
    for kel in mid:
        kelime_kok = kel[0][:kel[0].index("_")]
        kelime_len = len(kelime_kok)
        if kelime_kok.endswith("mak") or kelime_kok.endswith("mek"):
            kelime_len -= 3
        not_inserted = True
        for index in range(len(temp)):
            temp_kelime = temp[index]
            temp_kelime_kok = temp_kelime[0][:temp_kelime[0].index("_")]
            temp_len = len(temp_kelime_kok)
            if temp_kelime_kok.endswith("mak") or temp_kelime_kok.endswith("mek"):
                temp_len -= 3
            if(kelime_len>temp_len):
                temp.insert(index,kel)
                not_inserted = False
        if not_inserted:
            temp.append(kel)
    output = l+temp
    if len(output)==0:
        output.append([kelime+"_1","çaresiz",kelime+"_1",])
    return output

def checkSuffixValidation(suff):
    suffixList = ["","a", "abil", "acağ", "acak", "alım", "ama", "an", "ar", "arak", "asın", "asınız", "ayım", "da", "dan", "de", "den", "dı", "dığ", "dık", "dıkça", "dır", "di", "diğ", "dik", "dikçe", "dir", "du", "duğ", "duk", "dukça", "dur", "dü", "düğ", "dük", "dükçe", "dür", "e", "ebil", "eceğ", "ecek", "elim", "eme", "en", "er", "erek", "esin", "esiniz", "eyim", "ı", "ıl", "ım", "ımız", "ın", "ınca", "ınız", "ıp", "ır", "ıyor", "ız", "i", "il", "im", "imiz", "in", "ince", "iniz", "ip", "ir", "iyor", "iz", "k", "ken", "la", "lar", "ları", "ların", "le", "ler", "leri", "lerin", "m", "ma", "madan", "mak", "maksızın", "makta", "maktansa", "malı", "maz", "me", "meden", "mek", "meksizin", "mekte", "mektense", "meli", "mez", "mı", "mış", "mız", "mi", "miş", "miz", "mu", "muş", "mü", "muz", "müş", "müz", "n", "nın", "nız", "nin", "niz", "nun", "nuz", "nün", "nüz", "r", "sa", "se", "sı", "sın", "sınız", "sınlar", "si", "sin", "siniz", "sinler", "su", "sun", "sunlar", "sunuz", "sü", "sün", "sünler", "sünüz", "ta", "tan", "te", "ten", "tı", "tığ", "tık", "tıkça", "tır", "ti", "tiğ", "tik", "tikçe", "tir", "tu", "tuğ", "tuk", "tukça", "tur", "tü", "tüğ", "tük", "tükçe", "tür", "u", "ul", "um", "umuz", "un", "unca", "unuz", "up", "ur", "uyor", "uz", "ü", "ül", "ün", "üm", "ümüz", "ünce", "ünüz", "üp", "ür", "üyor", "üz", "ya", "yabil", "yacağ", "yacak", "yalım", "yama", "yan", "yarak", "yasın", "yasınız", "yayım", "ydı", "ydi", "ydu", "ydü", "ye", "yebil", "yeceğ", "yecek", "yelim", "yeme", "yen", "yerek", "yesin", "yesiniz", "yeyim", "yı", "yım", "yın", "yınca", "yınız", "yıp", "yız", "yi", "yim", "yin", "yince", "yiniz", "yip", "yiz", "yken", "yla", "yle", "ymış", "ymiş", "ymuş", "ymüş", "yor", "ysa", "yse", "yu", "yum", "yun", "yunca", "yunuz", "yup", "yü", "yuz", "yüm", "yün", "yünce", "yünüz", "yüp", "yüz"]
    validList = []
    if suff in suffixList:
        validList.append(suff)
    for ind in range(1,len(suff)):
        if(suff[:ind] in suffixList):
            cont, contList = checkSuffixValidation(suff[ind:])
            if cont:
                contList = [suff[:ind]+"+"+l for l in contList]
                validList = validList+contList
    return len(validList)>0,validList

try:
    with open('zemberek/revisedDict.pkl', 'rb') as f:
        revisedDict = pickle.load(f)
except IOError:
    print("Please run trainLexicon.py to generate revisedDict.pkl file")

def lemma(text):
    words = nltk.word_tokenize(text)
    roots = []
    for word in words:
        #print("Possible lemmas for",word,"in ranked order:")
        root = findPos(word, revisedDict)[0][0][:-2]
        #print(root)
        roots.append(root)
    return ' '.join(roots)

tweets_df['processed14'] = tweets_df['processed13'].apply(lemma)
tweets_df['processed14'].to_csv('processed14stemmed.csv', header=False, index=False)
tweets_df

<ipython-input-20-c52338cd2e1d>:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['processed14'] = tweets_df['processed13'].apply(lemma)


,Datetime,Text,processed1,processed2,processed3,processed4,languages,processed5,processed6,processed7,processed8,processed9,processed10,processed11,processed12,processed13,processed14
17,2020-10-23 17:50:59+03:00,"@baskanzorluoglu Başkanım, olası bir deprem iç...","Başkanım, olası bir deprem için trabzonda ac...","Başkanım, olası bir deprem için trabzonda ac...","Başkanım, olası bir deprem için trabzonda ac...","başkanım , olası bir deprem için trabzonda aci...",tr,"başkanım , olası bir deprem için trabzonda aci...",başkanım olası bir deprem için trabzonda aci...,başkanım olası bir deprem için trabzonda acil ...,başkanım olası bir deprem için trabzonda acil ...,başkanım olası bir deprem için trabzonda acil ...,başkanım olası bir deprem için trabzonda acil ...,başkanım olası bir deprem için trabzonda acil ...,başkanım olası bir deprem için trabzonda acil ...,başkanım olası deprem trabzonda acil toplanma ...,başkan olası deprem trabzon acil topla alan ne...
68,2020-10-23 19:39:06+03:00,"Değerli arkadaşlar, nasıl ki ağır bir deprem b...","Değerli arkadaşlar, nasıl ki ağır bir deprem b...","Değerli arkadaşlar, nasıl ki ağır bir deprem b...","Değerli arkadaşlar, nasıl ki ağır bir deprem b...","değerli arkadaşlar , nasıl ki ağır bir deprem ...",tr,"değerli arkadaşlar , nasıl ki ağır bir deprem ...",değerli arkadaşlar nasıl ki ağır bir deprem ...,değerli arkadaşlar nasıl ki ağır bir deprem be...,değerli arkadaşlar nasıl ki ağır bir deprem be...,değerli arkadaşlar nasıl ki ağır bir deprem be...,değerli arkadaşlar nasıl ki ağır bir deprem be...,değerli arkadaşlar nasıl ki ağır bir deprem be...,değerli arkadaşlar nasıl ağır bir deprem bekle...,değerli arkadaşlar ağır deprem beklentisi içer...,değer arkadaş ağır deprem beklenti içeri ol şe...
97,2020-10-23 20:20:29+03:00,"Deprem, sel vb. doğal afetlerin yanı sıra topl...","Deprem, sel vb. doğal afetlerin yanı sıra topl...","Deprem, sel vb. doğal afetlerin yanı sıra topl...","Deprem, sel vb. doğal afetlerin yanı sıra topl...","deprem , sel vb. doğal afetlerin yanı sıra top...",tr,"deprem , sel vb. doğal afetlerin yanı sıra top...",deprem sel vb doğal afetlerin yanı sıra top...,deprem sel vb doğal afetlerin yanı sıra toplum...,deprem sel vb doğal afetlerin yanı sıra toplum...,deprem sel vb doğal afetlerin yanı sıra toplum...,deprem sel vb doğal afetlerin yanı sıra toplum...,deprem sel vb doğal afetlerin yanı sıra toplum...,deprem sel doğal afetlerin yanı sıra toplumsal...,deprem sel doğal afetlerin yanı sıra toplumsal...,deprem sel doğal afet yan sıra toplumsal olay ...
157,2020-10-23 21:50:44+03:00,@ibbhaberleri Osuruktan teyyare anketi yapıyor...,Osuruktan teyyare anketi yapıyor troll hesap...,Osuruktan teyyare anketi yapıyor troll hesap...,Osuruktan teyyare anketi yapıyor troll hesap...,osuruktan teyyare anketi yapıyor troll hesap ....,tr,osuruktan teyyare anketi yapıyor troll hesap ....,osuruktan teyyare anketi yapıyor troll hesap ...,osuruktan teyyare anketi yapıyor troll hesap s...,osuruktan teyyare anketi yapıyor troll hesap s...,osuruktan teyyare anketi yapıyor troll hesap s...,osuruktan teyyare anketi yapıyor troll hesap s...,osuruktan teyyare anketi yapıyor troll hesap s...,osuruktan teyyare anketi yapıyor troll hesap s...,osuruktan teyyare anketi yapıyor troll hesap s...,osuruk te anket yapı trol hesap sonra başkan ö...
277,2020-10-24 01:27:00+03:00,@bikapiacildi Bilimsel bir gerçek sadece. Oras...,Bilimsel bir gerçek sadece. Orası 1513'ten b...,Bilimsel bir gerçek sadece. Orası 1513'ten b...,Bilimsel bir gerçek sadece. Orası 1513'ten b...,bilimsel bir gerçek sadece . orası 1513'ten be...,tr,bilimsel bir gerçek sadece . orası 1513'ten be...,bilimsel bir gerçek sadece orası 1513 ten be...,bilimsel bir gerçek sadece orası 1513 ten beri...,bilimsel bir gerçek sadece orası ten beri büy...,bilimsel bir gerçek sadece orası ten beri büy...,bilimsel bir gerçek sadece orası ten beri büy...,bilimsel bir gerçek sadece orası ten beri büy...,bilimsel bir gerçek sadece orası ten beri büyü...,

In [58]:
tweets_df[['remove_stops', 'rescue']]

,remove_stops,rescue
0,merhaba mansuroğlu mahallesinde oturan çiftiz ...,1
1,depremde evlerini kaybeden vatandaşlarımızla o...,1
2,yılı aşkın süredir travma bellek üzerine çalış...,1
3,değerli izmirliler depremzedeler iyi niyetle b...,1
4,minik elif hayatı boyunca eğitim masraflarını ...,1
...,...,...
53768,ege denizi merkezli büyüklüğünde meydana geldi...,0
53769,rafatatatata mir,0
53770,karşınızda artık eski reyyan yok hercai yeni b...,0
53771,depremi izmir urlada şiddetli uzun hissettik h...,0


# Classifiers

In [15]:
sample = tweets_df[['remove_shorts', 'rescue']]#.groupby('rescue').apply(lambda x: x.sample(1196))
#sample = sample.reset_index(drop=True)
sample

,remove_shorts,rescue
0,merhaba biz mansuroğlu mahallesinde oturan bir...,1
1,depremde evlerini kaybeden vatandaşlarımızla o...,1
2,yılı aşkın süredir travma bellek üzerine çalış...,1
3,değerli izmirliler depremzedeler için iyi niye...,1
4,minik elif hayatı boyunca tüm eğitim masraflar...,1
...,...,...
17187,özellikle izmir kuşadası çok etkilenmiş görünü...,0
17188,izmir çok şiddetli bir deprem yaşadık geçmiş o...,0
17189,ege bölgesinde meydana gelen deprem izmir mani...,0
17190,ege denizi merkezli büyüklüğünde meydana geldi...,0


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
train, valid = train_test_split(sample, random_state=24, test_size = 0.3)

train = train.reset_index()
valid = valid.reset_index()

train.shape, valid.shape, sample.shape

((10827, 3), (4641, 3), (15468, 2))

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

X_train = train['remove_shorts']   # '0' corresponds to Texts/Reviews
y_train = train['rescue']   # '1' corresponds to Label (1 - positive and 0 - negative)
X_test = valid['remove_shorts']
y_test = valid['rescue']

In [20]:
y_test.value_counts()

0    4135
1     506
Name: rescue, dtype: int64

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

nb = Pipeline([('vect', CountVectorizer()),
                ('clf', MultinomialNB())
               ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

accuracy 0.8806291747468218
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      4135
           1       0.46      0.48      0.47       506

    accuracy                           0.88      4641
   macro avg       0.70      0.70      0.70      4641
weighted avg       0.88      0.88      0.88      4641

[[3844  291]
 [ 263  243]]


In [24]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

accuracy 0.8954966602025426
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4135
           1       0.55      0.24      0.33       506

    accuracy                           0.90      4641
   macro avg       0.73      0.61      0.64      4641
weighted avg       0.87      0.90      0.88      4641

[[4035  100]
 [ 385  121]]


In [25]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

accuracy 0.8741650506356389
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      4135
           1       0.42      0.42      0.42       506

    accuracy                           0.87      4641
   macro avg       0.68      0.67      0.67      4641
weighted avg       0.87      0.87      0.87      4641

[[3847  288]
 [ 296  210]]


C:\Users\ali\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
predictions = pd.DataFrame(zip(X_test, y_pred, y_test), columns=['text', 'pred', 'test'])
[i for i in predictions[predictions['pred'] != predictions['test']].values]

[array(['feryal özge tekin yeal öğrencisi yaşında mansuroğlu mah sokak sercan tuğur ziraat bankası arkası bornova katta gökdeniz kaplan kadriye kaplan sakarya caddesi mansuroğlu mahallesi rıza bey apartmanı bayraklı',
        1, 0], dtype=object),
 array(['sahibi aranıyor urla kekliktepe dörtyolu iletişim contactnumber imagelink',
        1, 0], dtype=object),
 array(['tunç soyer konuştu binada yıkıntı var trafik nedeniyle ekipler ulaşmakta zorluk yaşıyor mümkün oldukça özel araç kullanmasınlar',
        1, 0], dtype=object),
 array(['tunç soyer başlıkta yardım kampanyamızı başlatıyoruz gıda yardımı üncüsü erzak paketi üncüsü hijyen paketi incisi uyku tulumu',
        0, 1], dtype=object),
 array(['deprem anında yanınızda olması gereken eşyaları lütfen eksiksiz şekilde hazırda bulundurun imagelink',
        0, 1], dtype=object),
 array(['öğrencim enkaz altında lütfen yapın', 0, 1], dtype=object),
 array(['dikkat hasarlı eve girme yolları tıkama ekiplere yol ver gereksiz yere telefonu k

In [31]:
sample.to_csv('sample.csv', index=False)

In [25]:
print(pd.__version__)

1.3.5


In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

In [3]:
import pandas as pd

preprocessed_tweets = pd.read_csv('preprocessed_tweets.csv', usecols=['Datetime', 'Text', 'remove_shorts'])
preprocessed_tweets

,Datetime,Text,remove_shorts
0,2020-10-23 17:28:34+03:00,#VanDepremi unutursam aldığım nefes bana haram...,unutursam aldığım nefes bana haram olsun kalbi...
1,2020-10-23 17:28:52+03:00,İstanbul'da bırakın çözüm üretmeyi daha beter ...,istanbul bırakın çözüm üretmeyi daha beter old...
2,2020-10-23 17:29:18+03:00,@nevsinmengu Onlar böyle yaptıkca halk daha du...,onlar böyle yaptıkça halk daha duyarlı oluyor ...
3,2020-10-23 17:29:26+03:00,@TuranDursunn Diyelim deprem oldu veya başka b...,diyelim deprem oldu veya başka bir afet oldu z...
4,2020-10-23 17:30:48+03:00,Bosna Hersek'de M 3.5 büyüklüğünde bir deprem ...,bosna hersek büyüklüğünde bir deprem kaydedild...
...,...,...,...
940254,2021-10-01 02:35:01+03:00,Akdeniz\nbüyüklük 1.9 ML (30 dk önce)\nderinli...,akdeniz büyüklük önce derinlik ekim
940255,2021-10-01 02:43:06+03:00,@BurcuyineBurcu Hergün deprem olsa 6.5. Hortum...,hergün deprem olsa hortum çıksa hergün işsiz k...
940256,2021-10-01 02:57:01+03:00,Ege Denizi\nbüyüklük 1.5 ML (17 dk önce)\nderi...,ege denizi büyüklük önce derinlik ekim
940257,2021-10-01 02:58:01+03:00,@Erdemzd46842091 @Emirhan2565 @oyle_sandim @Oz...,ülke kaç deprem geçirdi van elazığ ülke kaç ih...


In [5]:
rescue_words = ['afad', 'acil', 'yardım', 'kurtarma',
                'ihtiyaç', 'ölü','arama', 'akut', 
                'apartman', 'bina', 'yaralı', 'kayıp',
                'afet', 'göçük', 'felaket', 'yıkıldı',
                'hasar', 'kıyamet', 'yıkılmış']

preprocessed_tweets['label1'] = [1 if i else 0 for i in preprocessed_tweets['Text'].str.contains('|'.join(rescue_words))]
preprocessed_tweets['label2'] = [1 if i else 0 for i in preprocessed_tweets['remove_shorts'].str.contains('|'.join(rescue_words))]
preprocessed_tweets

,Datetime,Text,remove_shorts,label1,label2
0,2020-10-23 17:28:34+03:00,#VanDepremi unutursam aldığım nefes bana haram...,unutursam aldığım nefes bana haram olsun kalbi...,0,0
1,2020-10-23 17:28:52+03:00,İstanbul'da bırakın çözüm üretmeyi daha beter ...,istanbul bırakın çözüm üretmeyi daha beter old...,0,0
2,2020-10-23 17:29:18+03:00,@nevsinmengu Onlar böyle yaptıkca halk daha du...,onlar böyle yaptıkça halk daha duyarlı oluyor ...,0,0
3,2020-10-23 17:29:26+03:00,@TuranDursunn Diyelim deprem oldu veya başka b...,diyelim deprem oldu veya başka bir afet oldu z...,1,1
4,2020-10-23 17:30:48+03:00,Bosna Hersek'de M 3.5 büyüklüğünde bir deprem ...,bosna hersek büyüklüğünde bir deprem kaydedild...,0,0
...,...,...,...,...,...
940254,2021-10-01 02:35:01+03:00,Akdeniz\nbüyüklük 1.9 ML (30 dk önce)\nderinli...,akdeniz büyüklük önce derinlik ekim,0,0
940255,2021-10-01 02:43:06+03:00,@BurcuyineBurcu Hergün deprem olsa 6.5. Hortum...,hergün deprem olsa hortum çıksa hergün işsiz k...,0,0
940256,2021-10-01 02:57:01+03:00,Ege Denizi\nbüyüklük 1.5 ML (17 dk önce)\nderi...,ege denizi büyüklük önce derinlik ekim,0,0
940257,2021-10-01 02:58:01+03:00,@Erdemzd46842091 @Emirhan2565 @oyle_sandim @Oz...,ülke kaç deprem geçirdi van elazığ ülke kaç ih...,0,0


In [6]:
preprocessed_tweets['Datetime'] = pd.to_datetime(preprocessed_tweets['Datetime'])
preprocessed_tweets

,Datetime,Text,remove_shorts,label1,label2
0,2020-10-23 17:28:34+03:00,#VanDepremi unutursam aldığım nefes bana haram...,unutursam aldığım nefes bana haram olsun kalbi...,0,0
1,2020-10-23 17:28:52+03:00,İstanbul'da bırakın çözüm üretmeyi daha beter ...,istanbul bırakın çözüm üretmeyi daha beter old...,0,0
2,2020-10-23 17:29:18+03:00,@nevsinmengu Onlar böyle yaptıkca halk daha du...,onlar böyle yaptıkça halk daha duyarlı oluyor ...,0,0
3,2020-10-23 17:29:26+03:00,@TuranDursunn Diyelim deprem oldu veya başka b...,diyelim deprem oldu veya başka bir afet oldu z...,1,1
4,2020-10-23 17:30:48+03:00,Bosna Hersek'de M 3.5 büyüklüğünde bir deprem ...,bosna hersek büyüklüğünde bir deprem kaydedild...,0,0
...,...,...,...,...,...
940254,2021-10-01 02:35:01+03:00,Akdeniz\nbüyüklük 1.9 ML (30 dk önce)\nderinli...,akdeniz büyüklük önce derinlik ekim,0,0
940255,2021-10-01 02:43:06+03:00,@BurcuyineBurcu Hergün deprem olsa 6.5. Hortum...,hergün deprem olsa hortum çıksa hergün işsiz k...,0,0
940256,2021-10-01 02:57:01+03:00,Ege Denizi\nbüyüklük 1.5 ML (17 dk önce)\nderi...,ege denizi büyüklük önce derinlik ekim,0,0
940257,2021-10-01 02:58:01+03:00,@Erdemzd46842091 @Emirhan2565 @oyle_sandim @Oz...,ülke kaç deprem geçirdi van elazığ ülke kaç ih...,0,0


In [7]:
preprocessed_tweets = preprocessed_tweets[(
    preprocessed_tweets['Datetime']>pd.Timestamp('2020-10-30 14:51', tz='Europe/Istanbul')
    ) & (preprocessed_tweets['Datetime']<pd.Timestamp('2020-11-7', tz='Europe/Istanbul'))]
preprocessed_tweets

,Datetime,Text,remove_shorts,label1,label2
4345,2020-10-30 14:51:50+03:00,deprem copk buyuk,deprem çöp büyük,0,0
4346,2020-10-30 14:51:59+03:00,deprem oluyorrrrrrrrrrrrrr,deprem oluyorrrrrrrrrrrrrr,0,0
4347,2020-10-30 14:52:11+03:00,Bu nasıl deprem Allah'ım,nasıl deprem allahım,0,0
4348,2020-10-30 14:52:17+03:00,Berbat deprem oldu,berbat deprem oldu,0,0
4349,2020-10-30 14:52:18+03:00,İzmirde felaket bir deprem oluyor,izmirde felaket bir deprem oluyor,1,1
...,...,...,...,...,...
401605,2020-11-06 23:59:49+03:00,"@gllk_m @Arif_Ilave Halk sokaga çıkıyor,adamı ...",halk sokağa çıkıyor adamı buluyor adam zelzele...,0,0
401606,2020-11-06 23:59:49+03:00,hala deprem haberleri okuyup okuyup perişan ol...,hala deprem haberleri okuyup okuyup perişan ol...,0,0
401607,2020-11-06 23:59:51+03:00,Deprem için bir şeyler paylaşırken konumu Izmi...,deprem için bir şeyler paylaşırken konumu izmi...,0,0
401608,2020-11-06 23:59:57+03:00,Desteğe ihtiyaçı olacakların deprem güçlendirm...,desteğe ihtiyacı olacakların deprem güçlendirm...,1,0


In [8]:
preprocessed_tweets.to_csv('preprocessed_tweets2.csv', index=False)